In [1]:
import os
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

import time

In [4]:
DATA_PATH = Path.cwd().parent.parent / "data" / "raw"

In [7]:
recipes_urls = pd.read_csv(DATA_PATH / "urls.csv")
recipes_urls = recipes_urls['url'].to_list()

## Display html of one page to see where to get the info

need to make https://www.veganrecipeclub.org.uk/recipes/15-minute-flourless-choc-chip-banana-muffins/ work

In [10]:
recipe = 'https://www.veganrecipeclub.org.uk/recipes/vegan-cherry-meringue-pie/'
index = recipes_urls.index(recipe)
index

485

In [12]:
page = requests.get(recipes_urls[index])
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

### Find the different section of the web page

In [15]:
#Title
soup.find_all("h1")[0].get_text(strip=True)

'Vegan Cherry Meringue Pie'

In [16]:
soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

[<p style="text-align: center; font-weight: bold;">Preparation time</p>,
 <p style="text-align: center; font-weight: bold;">Cook time</p>,
 <p style="text-align: center; font-weight: bold;">Total time</p>,
 <p style="text-align: center; font-weight: bold;">Meal</p>,
 <p style="text-align: center; font-weight: bold;">Servings</p>,
 <p style="text-align: center; font-weight: bold;">Ingredients</p>,
 <p style="text-align: center; font-weight: bold;">Instructions</p>,
 <p style="text-align: center; font-weight: bold;">Keywords</p>]

In [17]:
elements = dict()
sections = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
for i, section in enumerate(sections):
    elements[section.text] = i
elements

{'Preparation time': 0,
 'Cook time': 1,
 'Total time': 2,
 'Meal': 3,
 'Servings': 4,
 'Ingredients': 5,
 'Instructions': 6,
 'Keywords': 7}

### Inspect each element

In [26]:
#Preparation time
preparation_time = 0
cooking_time = 0
total_time = 0

html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

try:
    if(html_element[elements['Preparation time']].nextSibling.nextSibling.text != None):
        preparation_time = html_element[elements['Preparation time']].nextSibling.nextSibling.text
except:
    print("No preparation time found")
    
try:
    if(html_element[elements['Cook time']].nextSibling.nextSibling.text != None):
        cooking_time = html_element[elements['Cook time']].nextSibling.nextSibling.text
except:
    print("No cooking time found")

try: 
    if(html_element[elements['Total time']].nextSibling.nextSibling.text != None):
        total_time = html_element[elements['Total time']].nextSibling.nextSibling.text
except:
    print("No total time found")

print("Preparation time is", preparation_time)
print("Cooking time is", cooking_time)
print("Total time is", total_time)

Preparation time is  
Cooking time is  
Total time is 180


In [20]:
#Style of cooking
styles = soup.find_all("p", {"class": "intList"})
if len(styles) != 0:
    for style in styles:
        print(style.get_text(strip=True))

In [21]:
#Type of recipe
try:
    types = html_element[elements['Meal']].nextSibling.nextSibling.find_all("li")
    for type_ in types:
        print(type_.get_text())
except:
    print("Type of recipe not found")

vegan-desserts


In [23]:
#Number of servings
try:
    html_element[elements['Servings']].nextSibling.nextSibling.get_text()
except:
    print("Number of servings not found")

In [24]:
#Ingredients
try:
    HTML_ingredients = html_element[elements['Ingredients']].nextSibling.nextSibling
    for item in HTML_ingredients.find_all("li"):
        print(item.get_text())

    if(len(HTML_ingredients.find_all("li")) == 0):
        #Ingredients
        HTML_ingredients = HTML_ingredients[elements['Ingredients']].parent.find("div", {"class": "recipe-list"}).p
        for ingredient in HTML_ingredients.get_text().split("\n"):
            print(ingredient)
except:
    print("List of ingredients not found")

450g/3 cups plain (all-purpose) flour, plus extra for dusting
250g/1 cup vegan spread/margarine/butter
55g/¼ cup caster (superfine) sugar
185ml/¾ cup dairy-free milk of your choice
1kg/5 cups jarred morello or sour cherries, drained (juice reserved)
575g/2½ cups (firmly packed) light brown sugar
Zest of ½ lemon
2 tbsp cornflour (cornstarch), sieved
1 tbsp ground cinnamon
2 tsp mixed spice
125ml/½ cup aquafaba (chick pea liquid)
Pinch of citric acid
230g/1 cup caster (superfine) sugar


In [25]:
#Instructions
try: 
    HTML_instructions = html_element[elements['Instructions']].nextSibling.nextSibling
    for item in HTML_instructions.find_all("li"):
        print(item.get_text())
except:
    print("Instructions not found")

To make the pie base, combine all the ingredients, except the milk, in a bowl. Using your fingers, rub the butter into the dry ingredients until it resembles loose, wet sand.
Make a well in the centre and pour in the milk. Gently incorporate the milk into the flour until you have a firm dough, but be careful not to overwork it.
Wrap the dough in plastic wrap and refrigerate for at least 30 minutes.
Once the pastry has had time to rest, grease a 20cm (8 in) pie dish with cooking spray or some spread.
Lightly dust a surface with flour and roll out the pastry to about 4mm (¬π/8 in) thick, ensuring it is larger than the tart tin.
Gently roll the pastry back over your rolling pin, then unroll it over the top of the pie dish. Use your fingers to gently press the pastry into the dish.
Carefully trim any excess pastry with a small knife. Refrigerate for another 30 minutes.
Preheat the oven to 160°C (320°F).
Line the pastry with a piece of baking paper, ensuring it comes up the sides, and fill 